In [3]:
import pandas as pd
# from sklearn.preprocessing import StandardScaler
# from sklearn.decomposition import PCA
# from sklearn.cluster import KMeans
# Import necessary libraries
import torch
# import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D

from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import numpy as np
import matplotlib.pyplot as plt
import tokenizer
import json
import pickle
import pandas as pd
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from bertopic import BERTopic
from transformers import BertTokenizer, BertModel
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)
# Load the data
data = pd.read_csv('D:\proj\风来feng\______BERT__.csv')  ## 替换成你的文件

# Extract the text column
text_list = data['适合BERT聚类的格式总结']

# Load stopwords from the file
with open('停用词.txt', 'r', encoding='utf-8') as file:
    stopwords = set(file.read().splitlines())

# Function to remove stopwords from a text
def remove_stopwords(text, stopwords):
    return ' '.join([word for word in text.split() if word not in stopwords])

# Remove stopwords from the text_list
text_list = text_list.apply(lambda x: remove_stopwords(x, stopwords))

# Initialize the BERT tokenizer and model
model_name = "bert-base-uncased"


tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name).to(device)

# Tokenize the text list
inputs = tokenizer(text_list.to_list(), return_tensors="pt", padding=True, truncation=True)
inputs = {key: value.to(device) for key, value in inputs.items()}

# Compute embeddings
with torch.no_grad():
    outputs = model(**inputs)

# Get the last hidden state
last_hidden_state = outputs.last_hidden_state

# Extract sentence embeddings (using [CLS] token)
sentence_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()

# Initialize the BERTopic model
topic_model = BERTopic()

# Use the BERTopic model to predict topics for the embeddings
topics, probabilities = topic_model.transform(sentence_embeddings)

# Extract the topics and their keywords
topic_info = topic_model.get_topic_info()
keywords = topic_model.get_topics()

# Display the topics and their keywords
result_df = pd.DataFrame({
    '适合BERT聚类的格式总结': text_list,
    'Topic': topics,
    'Name': topic_info.set_index('Topic').loc[topics]['Name'].values
})
result_df['主题词1'] = [keywords[topic][0][0] if topic in keywords and len(keywords[topic]) > 0 else '' for topic in topics]
result_df['主题词2'] = [keywords[topic][1][0] if topic in keywords and len(keywords[topic]) > 1 else '' for topic in topics]
result_df['主题词3'] = [keywords[topic][2][0] if topic in keywords and len(keywords[topic]) > 2 else '' for topic in topics]

# Save the DataFrame to a CSV file
result_df.to_csv('BERT_topic_results.csv', index=False)

# Clear variables to free up memory
del data, text_list, inputs, outputs, last_hidden_state, sentence_embeddings, topics, probabilities, topic_info, keywords, result_df
torch.cuda.empty_cache()

print(topic_info)
for topic, words in keywords.items():
    print(f"Topic {topic}: {words}")


cpu


RuntimeError: [enforce fail at alloc_cpu.cpp:114] data. DefaultCPUAllocator: not enough memory: you tried to allocate 4410310656 bytes.

In [4]:
import torch
print(torch.cuda.is_available()) 

False
